# Figure 4

## Imports

In [ ]:
import copy
import matplotlib.pyplot as plt
import numpy

import dolfin_mech as dmech

## Parameters

### Mesh

In [ ]:
cube_params = {"path_and_mesh_name":"Meshes/generic_lung.xdmf"}

### Material

In [ ]:
params = {
    "alpha": 0.16, # MG: Units!
    "gamma":0.5, # MG: Units!
    "c1":1.2, # MG: Units! # MG: C'était 0.6 pour Fig3?
    "c2":0., # MG: Units!
    "kappa":1e2, # MG: Units!
    "eta":1e-5, # MG: Units!
    "rho_solid":1.06e-6} # MG: Units!

mat_params = {"scaling":"linear", "parameters":params}

### Loading

In [ ]:
pe, pi = -0.5, -2. # MG: Units!

g = 9.81e3 # MG: Units!

## Computing porosity distributions

In [ ]:
gravity_lst = [0, 1]
for gravity_ in gravity_lst:

    load_params_inverse = {
        "type":"p_boundary_condition0", "f":-gravity_*g, "P0":float(pe)}
    load_params_direct_exhalation = {
        "type":"p_boundary_condition", "f":-gravity_*g, "P0":float(pe)}
    load_params_direct_inhalation = {
        "type":"p_boundary_condition", "f":-gravity_*g, "P0":float(pi)}
    
    ### computing the unloaded configuration
    Uref, phisref_computation, Vexpiini, Vref = dmech.run_RivlinCube_PoroHyperelasticity(
        inverse=1,
        cube_params=cube_params,
        mat_params=mat_params,
        load_params=load_params_inverse,
        inertia=1,
        step_params={"dt_ini":1., "dt_min":1e-4},
        res_basename="Fig4-unloaded",
        get_results=1,
        verbose=1)

    ### computing the end-exhalation configuration
    phisref_imposed = [numpy.random.uniform(low=0.4, high=0.6) for i in range(len(phisref_computation))]
    if (gravity_ == 0): # MG: What is the difference between both?
        Uexhal, phisexhal_g0, Vunloaded, Vexhal = dmech.run_RivlinCube_PoroHyperelasticity(
            inverse=0,
            cube_params=cube_params,
            move_params={"move":True, "U":Uref},
            porosity_params={"type": "function_xml_from_array", "val":phisref_imposed},
            mat_params=mat_params,
            load_params=load_params_direct_exhalation,
            inertia=1,
            step_params={"dt_ini":0.125, "dt_min":1e-4},
            res_basename="Fig4-exhalation",
            get_results=1,
            verbose=1)
    else:
        Uexhal, phisexhal_g, Vunloaded, Vexhal = dmech.run_RivlinCube_PoroHyperelasticity(
            inverse=0,
            cube_params=cube_params,
            move_params={"move":True, "U":Uref},
            porosity_params={"type": "function_xml_from_array", "val":phisref_imposed},
            mat_params=mat_params,
            load_params=load_params_direct_exhalation,
            inertia=1,
            step_params={"dt_ini":0.125, "dt_min":1e-4},
            res_basename="Fig4-exhalation",
            get_results=1,
            verbose=1)

    ### computing the end-inhalation configuration
    Uinhal, phisinhal, Vunloaded, Vinhal = dmech.run_RivlinCube_PoroHyperelasticity(
        inverse=0,
        cube_params=cube_params,
        move_params={"move":True, "U":Uref},
        porosity_params={"type": "function_xml_from_array", "val":phisref_imposed},
        mat_params=mat_params,
        load_params=load_params_direct_inhalation,
        inertia=1,
        step_params={"dt_ini":0.125, "dt_min":1e-4},
        res_basename="Fig4-inhalation",
        get_results=1,
        verbose=1)

    if (gravity_ == 0):
        phisinhal_g0 = copy.deepcopy(phisinhal)

In [ ]:
porosity_lst = numpy.linspace(0, 1, 300)

porosity_exhal_g0 = []
porosity_inhal_g0 = []
porosity_exhal_g  = []
porosity_inhal_g  = []
porosity_ref      = []
porosity_plot     = []
for c in range(0, len(porosity_lst)-1):
    min = porosity_lst[c]
    max = porosity_lst[c+1]
    porosity_exhal_g0.append(numpy.sum([min<=i<max for i in phisexhal_g0]))
    porosity_inhal_g0.append(numpy.sum([min<=i<max for i in phisinhal_g0]))
    porosity_exhal_g.append(numpy.sum([min<=i<max for i in phisexhal_g]))
    porosity_inhal_g.append(numpy.sum([min<=i<max for i in phisinhal]))
    porosity_ref.append(numpy.sum([min<=i<max for i in phisref_imposed]))
    porosity_plot.append(1-(min+(max-min)/2))

## Generating plots

### End-exhalation

In [ ]:
fig, ax = plt.subplots()
width=max-min
ax.bar(porosity_plot, porosity_exhal_g0, color="blue", label="w/o gravity", alpha=0.5, edgecolor="black", width=width)
ax.bar(porosity_plot, porosity_exhal_g, color="orange", label="w gravity", alpha=0.5, edgecolor="black", width=width)
ax.bar(porosity_plot, porosity_ref, color="green", label="reference", alpha=0.5, edgecolor="black", width=width)
ax.set_ylabel("Frequency")
ax.set_title("Porosity")
ax.legend()
plt.xlim([0.3, 0.9])
plt.show()

### End-inhalation

In [ ]:
fig, ax = plt.subplots()
width=max-min
ax.bar(porosity_plot, porosity_inhal_g0, color="blue", label="w/o gravity", alpha=0.5, edgecolor="black", width=width)
ax.bar(porosity_plot, porosity_inhal_g, color="orange", label="w gravity", alpha=0.5, edgecolor="black", width=width)
ax.bar(porosity_plot, porosity_ref, color="green", label="reference", alpha=0.5, edgecolor="black", width=width)
ax.set_ylabel("Frequency")
ax.set_title("Porosity")
ax.legend()
plt.xlim([0.3, 0.9])
plt.show()